# Description

* Creating OTUs with usearch 
* Taxonomy assignment with QIIME using SILVA database version 128.
* Filtering out Chloroplasts and Mitochondria with Mothur.

# Setting variables

In [18]:
import os

workDir = '/home/sam/notebooks/hemp_microbiome/data/16S_OTUs/'
qcDir = '/home/sam/notebooks/hemp_microbiome/data/16S_OTUs/'
qcFinal = os.path.join(qcDir, 'QC', 'finalQC.fasta')

dbDir = '~/databases/SILVA.128/SILVA_128_QIIME_release/'
dbSeqs = os.path.join(dbDir, 'rep_set/rep_set_16S_only/97/97_otus_16S.fasta')
dbTax = os.path.join(dbDir, 'taxonomy/16S_only/97/consensus_taxonomy_7_levels.txt')

nprocs = 20

# Init

In [19]:
import os
import sys
import glob
import pandas as pd
#from cogent.parse.fasta import MinimalFastaParser as parse
from Bio import SeqIO
%load_ext rpy2.ipython
#%load_ext pushnote

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [20]:
%%R
library(ggplot2)
library(dplyr)
library(tidyr)
library(gridExtra)

In [21]:
if not os.path.isdir(workDir):
    os.makedirs(workDir)
%cd $workDir

/home/sam/notebooks/hemp_microbiome/data/16S_OTUs


### Symlinking qc'ed seq file

In [22]:
!ln -s $qcFinal .

In [23]:
qcFinal = os.path.split(qcFinal)[1]
!ls -thlc $qcFinal

lrwxrwxrwx 1 sam sam 66 Apr 23 14:49 finalQC.fasta -> /home/sam/notebooks/hemp_microbiome/data/16S_OTUs/QC/finalQC.fasta


# Just unique sequences

In [24]:
cmd =  'mothur "#unique.seqs(fasta={})"'.format(qcFinal)
!$cmd | tail -n 30

12117000	4304974
12118000	4305862
12119000	4306765
12120000	4307666
12121000	4308559
12122000	4309468
12123000	4310360
12124000	4311269
12125000	4312154
12126000	4313044
12127000	4313936
12128000	4314834
12129000	4315746
12130000	4316694
12131000	4317628
12132000	4318581
12133000	4319531
12134000	4320481
12135000	4321436
12136000	4322386
12137000	4323342
12138000	4324303
12138757	4325049

Output File Names: 
finalQC.names
finalQC.unique.fasta


mothur > quit()


In [25]:
qcFinalUniq = os.path.splitext(qcFinal)[0] + '.unique.fasta'

ret = !grep -c ">" $qcFinalUniq
print ('Number of unique sequences: {}'.format(ret[0]))

Number of unique sequences: 4325049


# Formatting seq names for usearch

In [26]:
qcFinalName = os.path.splitext(qcFinal)[0] + '.names'

counts = {}
with open(qcFinalName) as iFH:
    for line in iFH:
        seedID, seqIDs = line.split("\t")
        count = len(seqIDs.split(","))
        counts[seedID] = count 

In [27]:
qcFinalUs = os.path.splitext(qcFinalUniq)[0] + '.usearch_names.fasta'

with open(qcFinalUs, 'w') as oFH:
    for entry in SeqIO.parse(open(qcFinalUniq),'fasta'):
        #n, s = fasta.id, fasta.seq.tostring()
        n, s = entry.id, entry.seq.tostring()
        if counts[n] > 1:
            oFH.write(">%s;size=%s;\n%s\n"%(n,counts[n],s))
        else:
            continue

/home/sam/anaconda3/lib/python3.6/site-packages/Bio/Seq.py:345: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [28]:
#!cd $workDir; \
!head -n 6 $qcFinalUs

>3-1-RT_77;size=12;
TCCAGAGGATGCAAGCGTTATCCGGAATTATTGGGCGTAAAGCGTCTGTAGGTGGCTTTTTAAGTCCGCCGTCAAATCCCAGGGCTCAACCCTGGACAGGCGGTGGAAACTACCAAGCTGGAGTACGGTAGGGGCAGAGGGAATTTCCGGTGGAGCGGTGAAATGCGTAGAGATCGGAAAGAACACCAACGGCGAAAGCACTCTGCTGGGCCGACACTGACACTGAGAGACGAAAGCTAGGGGAGCGAATGGG
>4-3-F_96;size=56;
TCCGTAGGTGGCAAGCGTTATCCGGAATTATTGGGCGTAAAGCGCGCGCAGGTGGTTTCTTAAGTCTGATGTGAAAGCCCACGGCTCAACCGTGGAGGGTCATTGGAAACTGGGAGACTTGAGTGCAGAAGAGGAAAGTGGAATTCCATGTGTAGCGGTGAAATGCGTAGAGATATGGAGGAACACCAGTGGCGAAGGCGACTTTCTGGTCTGTAACTGACACTGAGGCGCGAAAGCGTGGGGAGCAAACAGG
>6-1-F_101;size=28;
TACAGAGGATGCAAGCGTTATCCGGAATGCTTGGGCGTAACGCGTCTGTAGGTGGCTTTTTAAGTCCGCCGTCAAATCCCAGGGCTCAACCCTGGACAGGCGGTGGAAACTACCAAGCTGGAGTACGGTAGGGGCAGAGGGAATTTCCGGTGGAGCGGTGAAATGCGTAGAGATCGGAAAGAACACCAACGGCGAAAGCACTCTGCTGGGCCGACACTGACACTGAGAGACGAAAGCTAGGGGAGCGAATGGG


# Usearch pipeline

In [29]:
qcFinalUsSort = os.path.splitext(qcFinalUs)[0] + '_sorted.fasta'
cmd = 'usearch -sortbysize {} -fastaout {} -minsize 2'.format(qcFinalUs, qcFinalUsSort)
!$cmd 

usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

00:02 226Mb   100.0% Reading finalQC.unique.usearch_names.fasta
00:02 193Mb  Getting sizes                                     
00:02 198Mb  Sorting 609200 sequences
00:04 200Mb   100.0% Writing output


In [30]:
!head $qcFinalUsSort
!echo 
!tail $qcFinalUsSort

>4-1-F_111;size=1829729;
TACAGAGGATGCAAGCGTTATCCGGAATGATTGGGCGTAAAGCGTCTGTAGGTGGCTTTTTAAGTCCGCCGTCAAATCCC
AGGGCTCAACCCTGGACAGGCGGTGGAAACTACCAAGCTGGAGTACGGTAGGGGCAGAGGGAATTTCCGGTGGAGCGGTG
AAATGCGTAGAGATCGGAAAGAACACCAACGGCGAAAGCACTCTGCTGGGCCGACACTGACACTGAGAGACGAAAGCTAG
GGGAGCGAATGGG
>5-5-RT_650;size=623452;
GACGGGGGGGGCAAGTGTTCTTCGGAATGACTGGGCGTAAAGGGCACGTAGGCGGTGAATCGGGTTGAAAGTGAAAGTCG
CCAAAAACTGGTGGAATGCTCTCGAAACCAATTCACTTGAGTGAGACAGAGGAGAGTGGAATTTCGTGTGTAGGGGTGAA
ATCCGGAGATCTACGAAGGAACGCCAAAAGCGAAGGCAGCTCTCTGGGTCCCTACCGACGCTGGAGTGCGAAAGCATGGG
GAGCGAACGGG

>3-2-RT_4113442;size=2;
TACGTAGGGTGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTTGTAGGTGGCTCGTTGCGTCGGGAGTGAAAACCC
GTGGCTTAACCACGGGACTGCTTCCGATACGGGCGAGCTAGAGTTCGGCAGGGGAGACTGGAATTCCTGGTGTAGCGGTG
GAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGTTCTCTGGTCCGGTACTGACGCTGAGGTGCGAAAGCGTG
GGGAGCAAACAGG
>3-4-BS_4113452;size=2;
GACGAACCGAGCGAACGTTGTTCGGAATCACTGGGCTTAAAGGGCGCGTAGGCGGGTCAGCAAGTCCGGGGTGAAATCTT
TCGGCTCAACCGGAAAAGTGCCTTGGATACTGCTGATCTGGAGGGAGGTAG

In [31]:
otuFile = 'otus.fasta'
cmd = 'usearch -cluster_otus {} -otus {}'.format(qcFinalUsSort, otuFile)
!$cmd 

usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

20:28 305Mb   100.0% 16604 OTUs, 144170 chimeras, 107803 chimeras


In [32]:
otuFileRn = os.path.splitext(otuFile)[0] + 'n.fasta'

In [33]:
%%bash -s "$otuFile" "$otuFileRn"

 bioawk -c fastx '{print ">" "OTU" "." NR "\n" $seq}' $1 > $2

In [35]:
!head -n 4 $otuFileRn

>OTU.1
TACAGAGGATGCAAGCGTTATCCGGAATGATTGGGCGTAAAGCGTCTGTAGGTGGCTTTTTAAGTCCGCCGTCAAATCCCAGGGCTCAACCCTGGACAGGCGGTGGAAACTACCAAGCTGGAGTACGGTAGGGGCAGAGGGAATTTCCGGTGGAGCGGTGAAATGCGTAGAGATCGGAAAGAACACCAACGGCGAAAGCACTCTGCTGGGCCGACACTGACACTGAGAGACGAAAGCTAGGGGAGCGAATGGG
>OTU.2
GACGGGGGGGGCAAGTGTTCTTCGGAATGACTGGGCGTAAAGGGCACGTAGGCGGTGAATCGGGTTGAAAGTGAAAGTCGCCAAAAACTGGTGGAATGCTCTCGAAACCAATTCACTTGAGTGAGACAGAGGAGAGTGGAATTTCGTGTGTAGGGGTGAAATCCGGAGATCTACGAAGGAACGCCAAAAGCGAAGGCAGCTCTCTGGGTCCCTACCGACGCTGGAGTGCGAAAGCATGGGGAGCGAACGGG


### Assigning taxonomy

In [36]:
otuFileTax = os.path.splitext(otuFileRn)[0] + '_tax'
!assign_taxonomy.py -r $dbSeqs -t $dbTax -i $otuFileRn -o $otuFileTax

### Removing Chloroplast, Eukaryal and mitochodria sequences

In [37]:
otuTax = os.path.join(otuFileTax, 'otusn_tax_assignments.txt')

to_rm = 'Chloroplast|Eukaryota|Unassigned|Mitochondria'
to_rm_file = 'to_remove_tax.accnos'

awk_cmd = '{print \$1}'
cmd = 'egrep "{}" {} | awk "{}" > {}'.format(to_rm, otuTax, awk_cmd, to_rm_file)
!$cmd

In [38]:
ret = !wc -l $to_rm_file
print ('Number of lines: {}'.format(ret[0]))
!head $to_rm_file

Number of lines: 1491 to_remove_tax.accnos
OTU.11583
OTU.11581
OTU.5688
OTU.1960
OTU.13285
OTU.13284
OTU.12137
OTU.5060
OTU.13752
OTU.7833


In [39]:
cmd = 'mothur "#remove.seqs(fasta={}, accnos={})"'.format(otuFileRn, to_rm_file)
!$cmd | tail -n 30

Last updated: 3/20/2017

by
Patrick D. Schloss

Department of Microbiology & Immunology
University of Michigan
http://www.mothur.org

When using, please cite:
Schloss, P.D., et al., Introducing mothur: Open-source, platform-independent, community-supported software for describing and comparing microbial communities. Appl Environ Microbiol, 2009. 75(23):7537-41.

Distributed under the GNU General Public License

Type 'help()' for information on the commands that are available

For questions and analysis support, please visit our forum at https://www.mothur.org/forum

Type 'quit()' to exit program



mothur > remove.seqs(fasta=otusn.fasta, accnos=to_remove_tax.accnos)
Removed 1491 sequences from your fasta file.

Output File Names: 
otusn.pick.fasta


mothur > quit()


In [41]:
otuFilePick = os.path.splitext(otuFileRn)[0] + '.pick.fasta'

ret = !grep -c ">" $otuFileRn
print ('Pre-filter: number of sequences: {}'.format(ret[0]))

ret = !grep -c ">" $otuFilePick
print ('Post-filter: number of sequences: {}'.format(ret[0]))

Pre-filter: number of sequences: 16604
Post-filter: number of sequences: 15113


# Mapping reads

In [42]:
cmd_perl = 's/^>(.+)(_[^_]+)\n\$/>\$1\$2\_\$.;barcodelabel=\$1\n/' 

seqFile = os.path.splitext(qcFinal)[0] + '_usearchfmt.fasta'

cmd = 'perl -pe "{}" {} > {}'.format(cmd_perl, qcFinal, seqFile)
!$cmd

In [43]:
!head -n 6 $seqFile

>2-1-RS_1_1;barcodelabel=2-1-RS
TCCGTCGGCTGCCCGCGTTGTCCGGAATTCTTGGGCGTAAAGGGCTCGTAGGCGGTTTGTCCCGTCCGGAGTGAAAACTCAGGGCTTAACCCTGACCCTGCTTCCGATACGGGCAGACTAGAGGTATGCAGGGGAGAACGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCGGTGGCGAAGGCGGTTCTCTGGGCATTACCTGACGCTGAGGAGCGAAAGAGTGGGGAGCGAACAGG
>2-5-RT_3_3;barcodelabel=2-5-RT
TCCCGCGGTTGCACGCGTTATCCGGAATGATTGGGCGTAAAGCGTCTGTAGGTGGCTTTTTAAGTCCGCCGTCAAATCCCAGGGCTCAACCCTGGACAGGCTGTGGAAACTACCAAGCTGGAGTACGGTAGCGGCAGAGGGAATTTCCGGTGTAGCGGTGAAATGCGTAGATATCGGAAAGAACACCAACGGCGAAAGCACTCTGCTGGGCCGACACTGACACTGAGAGACGAAAGCTAGGGGAGCGAATGGG
>3-1-RT_4_5;barcodelabel=3-1-RT
TCCCTCGGCTCCACGCGTTGTTCGGCATTACTGGGCGTCAAGCGCACGTAGGCGGCTATTCAAGTCTGCGGTGAAAGCCCGGGGCTCAACCCCGGAACTGCCCTCGAAACTAGGTAGCTAGAATCTTGGAGAGGTCAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAAGAACACCAGTGGCGAAGGCGACTGACTGGACAAGTAATGACGCTGAGGTGCGAAAGCGAGGGGAGCAAACAGG


In [44]:
#see how many Gb this file is.
!du -h $seqFile

3.4G	finalQC_usearchfmt.fasta


In [45]:
#spliting file because it is too big
!pyfasta split -n 5 $seqFile


creating new files:
finalQC_usearchfmt.0.fasta
finalQC_usearchfmt.1.fasta
finalQC_usearchfmt.2.fasta
finalQC_usearchfmt.3.fasta
finalQC_usearchfmt.4.fasta


In [46]:
g = os.path.join(workDir+"QC", 'finalQC_usearchfmt.*.fasta')
fileList = glob.glob(g)
fileList 

['/home/sam/notebooks/hemp_microbiome/data/16S_OTUs/QC/finalQC_usearchfmt.4.fasta',
 '/home/sam/notebooks/hemp_microbiome/data/16S_OTUs/QC/finalQC_usearchfmt.3.fasta',
 '/home/sam/notebooks/hemp_microbiome/data/16S_OTUs/QC/finalQC_usearchfmt.1.fasta',
 '/home/sam/notebooks/hemp_microbiome/data/16S_OTUs/QC/finalQC_usearchfmt.2.fasta',
 '/home/sam/notebooks/hemp_microbiome/data/16S_OTUs/QC/finalQC_usearchfmt.0.fasta']

In [47]:
# running usearch on each split file
for f in fileList:
    sys.stderr.write('Processing {}\n'.format(f))

    ff,_ = os.path.splitext(f)
    _,i = os.path.splitext(ff)
    uc = 'readmap{}.uc'.format(i.lstrip('.')) 

    !usearch \
        -usearch_global $f \
        -db $otuFilePick \
        -strand plus -id 0.97 \
        -uc $uc \
        -threads $nprocs

usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

00:00 44Mb    100.0% Reading otusn.pick.fasta


Processing /home/sam/notebooks/hemp_microbiome/data/16S_OTUs/QC/finalQC_usearchfmt.4.fasta


00:00 11Mb    100.0% Masking (fastnucleo)
00:00 12Mb    100.0% Word stats          
00:00 12Mb    100.0% Alloc rows
00:00 26Mb    100.0% Build index
00:28 264Mb   100.0% Searching finalQC_usearchfmt.4.fasta, 48.4% matched
usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

00:01 44Mb    100.0% Reading otusn.pick.fasta


Processing /home/sam/notebooks/hemp_microbiome/data/16S_OTUs/QC/finalQC_usearchfmt.3.fasta


00:01 11Mb    100.0% Masking (fastnucleo)
00:01 12Mb    100.0% Word stats          
00:01 12Mb    100.0% Alloc rows
00:01 26Mb    100.0% Build index
00:27 264Mb   100.0% Searching finalQC_usearchfmt.3.fasta, 48.4% matchedarching finalQC_usearchfmt.3.fasta, 50.7% matched
usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

00:00 44Mb    100.0% Reading otusn.pick.fasta


...

00:00 11Mb    100.0% Masking (fastnucleo)
00:00 12Mb    100.0% Word stats          
00:00 12Mb    100.0% Alloc rows
00:00 26Mb    100.0% Build index
00:28 264Mb   100.0% Searching finalQC_usearchfmt.1.fasta, 48.4% matched
usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

00:00 44Mb    100.0% Reading otusn.pick.fasta


Processing /home/sam/notebooks/hemp_microbiome/data/16S_OTUs/QC/finalQC_usearchfmt.2.fasta


00:00 11Mb    100.0% Masking (fastnucleo)
00:00 12Mb    100.0% Word stats          
00:00 12Mb    100.0% Alloc rows
00:01 26Mb    100.0% Build index
00:28 264Mb   100.0% Searching finalQC_usearchfmt.2.fasta, 48.3% matched
usearch v9.2.64_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: seb369@cornell.edu

00:00 44Mb    100.0% Reading otusn.pick.fasta
00:00 11Mb    100.0% Masking (fastnucleo)    


Processing /home/sam/notebooks/hemp_microbiome/data/16S_OTUs/QC/finalQC_usearchfmt.0.fasta


00:00 12Mb    100.0% Word stats
00:00 12Mb    100.0% Alloc rows
00:00 26Mb    100.0% Build index
00:30 264Mb   100.0% Searching finalQC_usearchfmt.0.fasta, 48.4% matched


In [48]:
ucFile = 'readmap.uc'
!cat readmap[0-9].uc > $ucFile

In [49]:
otu_table_file = 'otu_table.txt'
cmd = 'python2 /opt/edgar_python_scripts/uc2otutab.py {} > {}'.format(ucFile, otu_table_file)
!$cmd 

readmap.uc 100.0%   


In [50]:
# make sure no existing biome file
otu_biom_file = 'otu_table.biom'
if os.path.isfile(otu_biom_file):
    os.unlink(otu_biom_file)
    
cmd = 'biom convert -i {} -o {} --table-type "OTU table" --to-json'.format(otu_table_file, otu_biom_file)
!$cmd

In [51]:
# make sure no existing biome file
otu_sum_file = 'otu_table_summary.txt'
if os.path.isfile(otu_sum_file):
    os.unlink(otu_sum_file)
    
cmd = 'biom summarize-table -i {} -o {}'.format(otu_biom_file, otu_sum_file)
!$cmd

In [52]:
!cat $otu_sum_file 

Num samples: 182
Num observations: 15089
Total count: 5902440
Table density (fraction of non-zero values): 0.087

Counts/sample summary:
 Min: 23.0
 Max: 232833.0
 Median: 26265.000
 Mean: 32430.989
 Std. dev.: 35174.482
 Sample Metadata Categories: None provided
 Observation Metadata Categories: None provided

Counts/sample detail:
F3-3: 23.0
M2-2: 33.0
6-1-BS: 34.0
F1-1: 38.0
6-5-RS: 40.0
F5-2: 43.0
M3-3: 43.0
M1-3: 44.0
M4-3: 49.0
3-4-RS: 58.0
F2-2: 63.0
6-3-F: 66.0
M4-1: 77.0
5-2-BS: 91.0
M1-1: 98.0
F1-2: 112.0
3-3-BS: 113.0
M4-2: 122.0
5-5-BS: 160.0
5-1-RT: 341.0
5-2-RT: 477.0
6-5-L: 731.0
4-3-RT: 868.0
3-3-F: 929.0
5-1-F: 1244.0
5-3-F: 1369.0
M1-2: 1526.0
F2-3: 1545.0
6-2-L: 1745.0
M2-1: 1782.0
M3-1: 1855.0
6-5-BS: 2464.0
4-4-F: 2718.0
2-4-RT: 2944.0
3-5-F: 3175.0
F4-1: 3267.0
3-5-RT: 3348.0
5-4-RS: 3409.0
5-5-RS: 3413.0
4-5-L: 3785.0
2-4-L: 4447.0
5-4-BS: 4710.0
F4-3: 4950.0
M3-2: 5393.0
3-3-L: 5514.0
5-2-RS: 5612.0
4-2-F: 6067.0
6-1-F: 6195.0
4-1-F: 6784.0
1-3-RT: 7141.0
F1-3: 

# Adding taxonomy

In [53]:
# make sure no existing biome file
otu_biom_tax_file = 'otu_table_wtax.biom'
if os.path.isfile(otu_biom_tax_file):
    os.unlink(otu_biom_tax_file)
    
# bio add-metadata
cmd = """biom add-metadata -i {} -o {} \
    --observation-metadata-fp {} \
    --sc-separated taxonomy \
    --float-fields consensus \
    --int-fields numhits \
    --observation-header OTUID,taxonomy,consensus,numhits"""
cmd = cmd.format(otu_biom_file, otu_biom_tax_file, otuTax)
!$cmd

In [54]:
!ls -thlc $otu_biom_tax_file

-rw-rw-r-- 1 sam sam 5.8M Apr 23 15:42 otu_table_wtax.biom


# Final files

In [55]:
!tail -n +2 $otu_table_file | wc -l | perl -pe 's/ .+//'

15089


In [56]:
print ('OTU table: {}'.format(otu_table_file))
print ('OTU pick file: {}'.format(otuFilePick))
print ('uclust formatted file: {}'.format(seqFile))

OTU table: otu_table.txt
OTU pick file: otusn.pick.fasta
uclust formatted file: finalQC_usearchfmt.fasta


In [57]:
# Number of OTUs
!tail -n +2 $otu_table_file | wc -l | perl -pe 's/ .+//'
!grep -c ">" $otuFilePick
!grep -c ">" $seqFile

15089
15113
12138757


In [58]:
# total number of samples
sys.stdout.write('Total number of samples: ')
!head -n 1 $otu_table_file | perl -pe 's/[^\t]+\t//; s/\t/\n/g' | wc -l

Total number of samples: 182
